In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


In [ ]:
!cp '/content/gdrive/MyDrive/pop_lupus.pkl.zip' .

In [ ]:
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.cluster import KMeans

In [ ]:
data = pickle.load(open('pop_lupus.pkl','rb'))
print(data.columns)

Index(['CD45', 'CD196_CCR6', 'CD181_CXCR1', 'HLA_DR', 'CD15', 'CD31_PECAM1',
       'CD8a', 'CD182_CXCR2', 'IgA', 'CD66ace', 'CD63', 'CD14', 'CD66b',
       'CD62L_Lselectin', 'CD3', 'CD27', 'CD86', 'CD10', 'CD197_CCR7', 'CD28',
       'CD11c', 'CD33', 'CD161', 'CD45RO', 'CD24', 'CD38', 'CD278_ICOS',
       'CD32', 'CD152_CTLA4', 'IgM', 'CD184_CXCR4', 'CD279_PD1', 'CD56',
       'CD16', 'Label_0.9', 'score', ' sample type', 'clusters'],
      dtype='object')


In [ ]:
populations = {}

for i in set(list(data['clusters'].values)):

    cluster = data.iloc[np.where(data['clusters'].values == i)[0]]
    cluster = cluster.reset_index()
    cluster = cluster.drop(['index'],1)

    if list(cluster['score'].values).count(0) > list(cluster['score'].values).count(1):
        diff = list(cluster['score'].values).count(0)-list(cluster['score'].values).count(1)
        cluster = cluster.drop(list(np.random.choice(np.where(cluster['score'] == 0)[0],diff,replace=False)),0)
    else:
        diff = list(cluster['score'].values).count(1)-list(cluster['score'].values).count(0)
        cluster = cluster.drop(list(np.random.choice(np.where(cluster['score'] == 1)[0],diff,replace=False)),0)
    ##balanced
    populations[i] = cluster 
        
        


In [ ]:
import pandas as pd
import copy as cp
cv = StratifiedKFold(n_splits = 5,shuffle=True)

allvals = []
for key,df in populations.items():

    binary_labels = df['score'].values
    marker_data = df.drop(['Label_0.9', 'score', ' sample type', 'clusters'],1)
    

    for col in marker_data.columns:
        if np.mean(marker_data[col].values) < 0.5:
            marker_data = marker_data.drop([col],1)
  

    if len(marker_data.columns) == 0:
        continue
    cols = marker_data.columns
    save_df = cp.deepcopy(marker_data)
    marker_data = marker_data.values
    result = 0
    all_coef = np.zeros(len(cols))
    for i, (train, test) in enumerate(cv.split(marker_data, binary_labels)):
        X,y = marker_data[train],binary_labels[train]
        X_test,y_test = marker_data[test],binary_labels[test]
        model = LogisticRegression().fit(X,y)
        preds = model.predict(X_test)
        result += accuracy_score(np.round(preds),y_test)
        all_coef+= np.exp(model.coef_[0])
        

    if result/5 > 0.74:
        odds = all_coef/5
        df2 = pd.DataFrame(odds, 
             cols, 
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)
        print(df2)
        break


            
train_d = save_df[['CD63']].values.reshape(-1,1)
for i, (train, test) in enumerate(cv.split(train_d, binary_labels)):
                X,y = train_d[train],binary_labels[train]
                X_test,y_test = train_d[test],binary_labels[test]
                model = LogisticRegression().fit(X,y)
                preds = model.predict(X_test)
                result += accuracy_score(np.round(preds),y_test)
                print(np.exp(model.coef_[0]))

 

                 coef
CD11c        2.655698
CD86         2.389356
CD38         2.007404
CD32         1.452904
CD31_PECAM1  1.334612
CD14         1.323004
CD8a         1.041301
HLA_DR       0.844392
CD184_CXCR4  0.671431
CD161        0.578626
CD45         0.487697
CD63         0.485732
CD33         0.382302
[0.58451206]
[0.59367501]
[0.5993601]
[0.58660899]
[0.59562822]


In [ ]:
data66 = data.iloc[np.where(data['CD66b'].values > 0.75)]
data66 = data66.reset_index()
data66 = data66.drop(['index'],1)
    
if list(data66[' sample type'].values).count('NDN') > list(data66[' sample type'].values).count('PBMC'):
        diff = list(data66[' sample type'].values).count('NDN') - list(data66[' sample type'].values).count('PBMC')
        data66 = data66.drop(list(np.random.choice(np.where(data66[' sample type'] == 'NDN')[0],diff,replace=False)),0)
else:
        diff = list(data66[' sample type'].values).count('PBMC') - list(data66[' sample type'].values).count('NDN')
        data66 = data66.drop(list(np.random.choice(np.where(data66[' sample type'] == 'PBMC')[0],diff,replace=False)),0)
    ##balanced
binary = data66[' sample type'].values

print(len(data66))

binary_labels = []
for i in binary:
    if i == 'NDN':
        binary_labels.append(0)
    else:
        binary_labels.append(1)
        
binary_labels = np.asarray(binary_labels)

marker_data = data66.drop(['Label_0.9', 'score', ' sample type', 'clusters'],1).values

cv = StratifiedKFold(n_splits = 5,shuffle=True)
result = 0
for i, (train, test) in enumerate(cv.split(marker_data,binary_labels)):
        X,y = marker_data[train],binary_labels[train]
        X_test,y_test = marker_data[test],binary_labels[test]
        model = LogisticRegression().fit(X,y)
        preds = model.predict(X_test)
        result += accuracy_score(np.round(preds),y_test)

print(result/5)
if result/5 > 0.74:
            odds = np.exp(model.coef_[0])
            df2 = pd.DataFrame(odds, 
            data66.drop(['Label_0.9', 'score', ' sample type', 'clusters'],1).columns, 
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)
            print(df2)

  

marker_data = data66.drop(['Label_0.9', 'score', ' sample type', 'clusters'],1)['CD66b'].values.reshape(-1,1)

cv = StratifiedKFold(n_splits = 5,shuffle=True)
result = 0
for i, (train, test) in enumerate(cv.split(marker_data,binary_labels)):
        X,y = marker_data[train],binary_labels[train]
        X_test,y_test = marker_data[test],binary_labels[test]
        model = LogisticRegression().fit(X,y)
        preds = model.predict(X_test)
        result += accuracy_score(np.round(preds),y_test)
print(model.coef_[0])


6311090
0.7441590913772422
                     coef
CD66b            7.433718
CD182_CXCR2      1.586399
CD184_CXCR4      1.146967
CD10             1.139034
CD181_CXCR1      1.106686
CD62L_Lselectin  1.092124
IgA              1.073194
CD24             1.057901
CD45RO           1.036478
CD16             1.034144
CD152_CTLA4      1.023113
CD86             1.022427
CD14             0.993772
CD27             0.973413
CD161            0.967858
CD279_PD1        0.967348
CD32             0.964649
CD28             0.956148
IgM              0.945571
CD38             0.939535
CD8a             0.920324
CD15             0.920028
CD278_ICOS       0.913114
HLA_DR           0.897165
CD56             0.892083
CD197_CCR7       0.865917
CD196_CCR6       0.849299
CD3              0.820691
CD31_PECAM1      0.819729
CD66ace          0.806422
CD45             0.769111
CD63             0.756692
CD33             0.714597
CD11c            0.696736
[1.69354301]
